# 检查pytorch的GPU是否可用

In [4]:
import torch

print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name())
print(torch.cuda.get_device_properties(0))

1.2.0+cu92
True
1
GeForce RTX 2080 SUPER
_CudaDeviceProperties(name='GeForce RTX 2080 SUPER', major=7, minor=5, total_memory=7979MB, multi_processor_count=48)


# tensor在某个通道上做平均

In [ ]:
import torch

In [10]:
x=torch.arange(16).view(2,2,2,2).to(float)
print(x)
x.pow(2).mean(1).view(x.size(0), -1)
# mean(1)是在通道维数上作平均

tensor([[[[ 0.,  1.],
          [ 2.,  3.]],

         [[ 4.,  5.],
          [ 6.,  7.]]],


        [[[ 8.,  9.],
          [10., 11.]],

         [[12., 13.],
          [14., 15.]]]], dtype=torch.float64)


tensor([[  8.,  13.,  20.,  29.],
        [104., 125., 148., 173.]], dtype=torch.float64)

# 设置flag增加运行效率

In [ ]:
import torch.backends.cudnn as cudnn
cudnn.benchmark = True

# 嵌套词典nested_dicted

In [15]:
from nested_dict import nested_dict

a = {"hi":{"you":{"dd": 3}}, "h":{"yo":{"d": 2}}}
for i in nested_dict(a).items_flat():
    print(i)

(('hi', 'you', 'dd'), 3)
(('h', 'yo', 'd'), 2)


In [16]:
b = {'.'.join(k): v for k, v in nested_dict(a).items_flat()}
print(b)

{'hi.you.dd': 3, 'h.yo.d': 2}


# 命令行参数

In [ ]:
import argparse

parser = argparse.ArgumentParser(description='Test')
parser.add_argument('--depth', default=16, type=int)
parser.add_argument('--width', default=1, type=float)
parser.add_argument('--dataset', default='CIFAR10', type=str)
parser.add_argument('--epochs', default=200, type=int, metavar='N',
                    help='number of total epochs to run')

args = parser.parse_args()  # 获取命令行参数
# args.depth

## BCE loss and BCEWithLogitsLoss

两者区别为输入是否经过sigmoid激活
https://discuss.pytorch.org/t/multi-label-multi-class-class-imbalance/37573

In [37]:
import torch
from torch import nn

In [38]:
x = torch.tensor([[0.1,0.8]], dtype=torch.float)
y = torch.tensor([[0,1]], dtype=torch.float)

使用标准的BCEWithLogitsLoss进行计算

In [41]:
cri_1 = nn.BCEWithLogitsLoss()
loss_1 = cri_1(x,y)
print(loss_1)

tensor(0.5577)


手动计算

In [30]:
sigmoid_x = nn.Sigmoid()(x)
print(sigmoid_x)
loss = (-torch.log(1-sigmoid_x[0][0]) -
          torch.log(sigmoid_x[0][1]))/2
print(loss)

tensor([[0.5250, 0.6900]])
tensor(0.5577)


假设第2类类别不均衡，pos=100,neg=300，所以正类的loss比重应该为300/100=3

In [34]:
pos_weight = torch.tensor([1,3], dtype=torch.float)
cri_2 = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
loss_2 = cri_2(x,y)
print(loss_2)

tensor(0.9288)


In [35]:
sigmoid_x = nn.Sigmoid()(x)
print(sigmoid_x)
loss = (-torch.log(1-sigmoid_x[0][0]) -
          3*torch.log(sigmoid_x[0][1]))/2
print(loss)

tensor([[0.5250, 0.6900]])
tensor(0.9288)


为不同的类添加不同的加权系数

In [47]:
cri_3 = nn.BCEWithLogitsLoss(reduction='none')
loss_3 = cri_3(x,y)
print(loss_3)
weights = torch.tensor([1,1], dtype=torch.float)
loss = (weights*loss_3).mean()
print(loss)

tensor([[0.7444, 0.3711]])
tensor(0.5577)


## 加载预训练模型

https://blog.csdn.net/weixin_41278720/article/details/80759933
**可以只加载模型的部分参数，见上面链接**

model.state_dict()中保存了{参数名:参数值}的字典

In [51]:
import torchvision.models as models

resnet34 = models.resnet34(pretrained=True)
resnet34.state_dict().keys()

In [ ]:
torch.save(model.state_dict(), PATH) # 保存模型为pth

In [ ]:
model = ModelClass()   # 需要先建立模型
model.load_state_dict(torch.load(PATH)) # 加载模型

## 建立自定义数据集

In [63]:
import torch
from torch.utils import data

class MyDataset(data.Dataset):
    def __init__(self):
        super(MyDataset, self).__init__()
        self.data = torch.randn(8,2)
    
    def __getitem__(self, index):
        return self.data[index], index
    
    def __len__(self):
        return self.data.size()[0]

In [64]:
data_set = MyDataset()
print(data_set.data)

tensor([[-1.3907, -0.0916],
        [-0.4626, -1.3323],
        [ 1.4242, -2.1718],
        [ 1.5850,  0.3320],
        [-1.0804,  0.3884],
        [ 0.6567, -0.1234],
        [ 1.6721, -0.7327],
        [-1.9595, -0.3512]])


In [67]:
data_loader = data.DataLoader(data_set, 
                              batch_size=4,
                              shuffle=False)
print(len(data_set))
for i, (number, labels) in enumerate(data_loader):
    print(number)
    print(labels)

8
tensor([[-1.3907, -0.0916],
        [-0.4626, -1.3323],
        [ 1.4242, -2.1718],
        [ 1.5850,  0.3320]])
tensor([0, 1, 2, 3])
tensor([[-1.0804,  0.3884],
        [ 0.6567, -0.1234],
        [ 1.6721, -0.7327],
        [-1.9595, -0.3512]])
tensor([4, 5, 6, 7])


## torch.stack的用法

In [73]:
a = torch.tensor([1,2])
b = torch.tensor([3,4])
c = torch.stack((a,b)).t()
print(c)

tensor([[1, 3],
        [2, 4]])


## 加载预训练模型

In [ ]:
from torchvision import models

model = models.resnet18(pretrained=True)
print(model)

In [ ]:
model = nn.Sequential(*list(model.children())[:-1])
print(model)